<a href="https://colab.research.google.com/github/Mohammadhsiavash/DeepL-Training/blob/main/AI%20Agents%20%2B%20Automation/04_AI_Calendar_Scheduler_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create an AI agent that:
1. Understands scheduling requests in natural language
2. Checks for available time slots
3. Suggests or inserts new events into a calendar
4. Uses tools like a calendar checker and event creator

# Tools Simulated
check_calendar_availability() : to retrieve free time slots

create_event() : to add an event

llm() : the reasoning agent using ReAct or tool-augmented prompts

# simulate calendar API

In [1]:
calendar = {
"2025-05-20 10:00": "Team Sync",
"2025-05-20 14:00": "Sales Demo",
"2025-05-21 09:00": "Weekly Standup"
}
def check_calendar_availability(date):
  hours = [f"{date} {str(h).zfill(2)}:00" for h in range(9, 18)]
  return [h for h in hours if h not in calendar]
def create_event(time_slot, title):
  calendar[time_slot] = title
  return f"✅ Event '{title}' scheduled at {time_slot}"


# Define Agent Prompt (ReAct Style)

In [3]:
SCHEDULER_PROMPT = """You are an AI Calendar Agent.
Respond to the user request by reasoning and using tools like `check_calendar_availability()` and `create_event()`.
Use the following format:
Task: <user request>
Thought: <your reasoning>
Action: <Python code>
Observation: <result>
... (repeat if needed)
Final Answer: <final message>
Task: {task}
"""


In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

def llm(prompt):
  messages = [
      {"role": "user", "content": prompt}
  ]
  text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True,
      enable_thinking=True # Switches between thinking and non-thinking modes. Default is True.
  )
  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  # conduct text completion
  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=32768
  )

  # Decode the entire generated output
  response_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True).strip()

  # Extract the part after the initial prompt
  # Assuming the response starts after the user prompt in the chat template
  # We need to find where the model's response begins
  # This might need adjustment based on the exact chat template output
  prompt_end_index = response_text.find(prompt) + len(prompt)
  if prompt_end_index != -1:
      model_response = response_text[prompt_end_index:].strip()
  else:
      model_response = response_text # Fallback if prompt not found (unlikely with chat template)


  # Further parse the model_response to get the content relevant to the agent
  # This is a basic attempt to extract based on keywords.
  # More robust parsing (e.g., regex) might be needed depending on model output.
  content = model_response

  # Remove the debug prints for raw IDs
  # print(f"Raw generated_ids: {generated_ids}") # Debug print
  # print(f"Extracted output_ids: {output_ids}") # Debug print

  return content

# Agent Simulation Loop

In [19]:
import re

def scheduler_agent(task):
  prompt = SCHEDULER_PROMPT.format(task=task)
  while True:
    print(prompt)
    # Replace the input() call below with a call to your llm() function
    # response = input("Agent Response (Thought/Action/Final):\n")
    response = llm(prompt) # <--- Call your LLM here
    print(f"Raw LLM response: {response}") # Print the raw response for debugging

    prompt += f"\n{response}"
    # Execute Action if present
    if "Action:" in response:
      action_code = response.split("Action:")[1].strip()
      try:
        result = eval(action_code)
      except Exception as e:
        result = str(e)
      prompt += f"\nObservation: {result}"
      if "Final Answer:" in response:
        break

# try it

In [20]:
scheduler_agent("Schedule a 30-minute project meeting tomorrow afternoon.")

You are an AI Calendar Agent.
Respond to the user request by reasoning and using tools like `check_calendar_availability()` and `create_event()`.
Use the following format:
Task: <user request>
Thought: <your reasoning>
Action: <Python code>
Observation: <result>
... (repeat if needed)
Final Answer: <final message>
Task: Schedule a 30-minute project meeting tomorrow afternoon.

Raw LLM response: assistant
<think>
Okay, the user wants to schedule a 30-minute project meeting tomorrow afternoon. Let me think about how to approach this.

First, I need to check if there's a calendar tool available. The user mentioned using the AI Calendar Agent, so I should use the `check_calendar_availability()` function to see if there's a slot for the meeting. That way, I can confirm if the meeting can be scheduled.

So, I'll call `check_calendar_availability()` with the event details: time tomorrow afternoon, duration 30 minutes. If there's an available slot, I can proceed to create the event using `crea